# Interferometry

The idea behind interferometry is that electromagnetic waves emitted from a certain point in the sky will cause characteristic interference between signals received by antennas. The reason is simply that the wave's travel time differs slightly between the antennas. With the period nature of the wave, this means that if we add up the measurements of both antennas, the signals will either sum up or cancel each other out.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import numpy as np
from ipywidgets import interact
plt.rcParams['figure.figsize'] = 16, 8
import sys
sys.path.append('../..')
from crocodile.simulate import *
from crocodile.synthesis import *
from util.visualize import *

# 1. Expected

Let's say that we are able to measure the interference pattern from two point source $(l_1,m_1)$ and $(l_2,m_2)$ at all points of a $(u,v)$ plane. In reality, this would require a thick mesh of receivers placed exactly perpendicular to the phase centre. This would show us a patterns that look as follows:

In [ ]:
N = 500; max_uv=25000
max_lm = 1/512/16; step_lm = 1/32768
bounds_lm = (-max_lm,max_lm,step_lm)
extent_lm = (-max_lm,max_lm,-max_lm,max_lm)
extent_uv = extent=(-max_uv,max_uv,-max_uv,max_uv)
def draw_uv(ax, s_split):
    ax.set_xlabel('u [$\lambda$]'); ax.set_ylabel('v [$\lambda$]')
    ax.set_title("Interference pattern")
    return ax.imshow(s_split.real, norm=colors.Normalize(vmin=-1, vmax=1), extent=extent_uv, origin='lower')
def draw_sky(ax, ls, ms):
    ax.set_title("Sky (around phase centre)")
    ax.set_xlabel('l [$1$]'); ax.set_ylabel('m [$1$]')
    ax.plot(ls, ms, 'ro')
    ax.set_xlim([-max_lm, max_lm])
    ax.set_ylim([-max_lm, max_lm])
def show_pattern_2(l1,m1, l2,m2):
    # Generate uvw coordinates
    v,u = max_uv*2*coordinates2(N)
    w = np.full(u.shape, 0, dtype=float)
    uvw = np.concatenate(np.transpose((u,v,w)))
    # Simulate for a full uv plane
    s = simulate_point(uvw, l1, m1)
    s += simulate_point(uvw, l2, m2)
    fig = plt.figure()
    draw_sky(fig.add_subplot(121), [l1,l2], [m1,m2])
    im = draw_uv(fig.add_subplot(122), np.array(np.split(s, N)))
    fig.colorbar(im,shrink=.4,pad=0.025)
    plt.show()    
interact(show_pattern_2,l1=bounds_lm, m1=bounds_lm,l2=bounds_lm, m2=bounds_lm);

Note how small changes to $l$ or $m$ cause large-scale shifts in the interference pattern: This is why long baselines are required to identify small structures on the sky.

We give $u$ and $v$ coordinates wavelengths here. For orientation: The longest baselines for Low (at 80 kilometre) will be correspond to 13.3 to 93.4 kilowavelenghts, whereas the longest planned baselines for Mid (at 150 kilometres) will be between 175.1 kilowavelenghts (Mid1) and 6904.8 kilowavelengths (Mid5C).

## 2. Inversion

The interaction between interference patterns from different directions is non-destructive, which means that we can invert the transformation shown above. In fact, this is a type of fourier transform, which is well-known to be invertible. So given enough data, we should be able to reconstruct the exact radiation intensity that we received from every sky direction.

However, this is a lot trickier in practice: We cannot actually measure inteference patterns in a plane as suggested above. Instead, we can only measure it at relatively fixed points given by the telescope positions. Every baseline has a characteristic reception pattern: